In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from TrialSolution import TrialSolution

/home/raroog/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def call(self, X):
  with tf.GradientTape() as tape:
    X = tf.convert_to_tensor(X)
    response = self.hidden_layer(X)
    response = self.output_layer(response)
    X1 = tf.concat([tf.reshape(X[:,0], shape=(X.shape[0], 1)),
                      tf.constant(1.0, dtype='float64', shape=(X.shape[0], 1))], axis=1)
    tape.watch(X1)
    response1 = self.hidden_layer(X1)
    response1 = self.output_layer(response1)
  der_resp1 = tape.gradient(response1, X1)
  der_resp1 = tf.reshape(der_resp1[:,1], shape=(response.shape[0],1))
  x = tf.reshape(X[:,0], shape=(response.shape[0],1))
  y = tf.reshape(X[:,1], shape=(response.shape[0],1))
  one = tf.constant(1., dtype='float64', shape=(response.shape[0],1))
  two = tf.constant(2., dtype='float64', shape=(response.shape[0],1))
  pi = tf.constant(np.pi, dtype='float64', shape=(response.shape[0],1))
  response -= response1
  response -= der_resp1
  response *= x*(one-x)*y
  response += y * two * tf.sin(pi * x)
  return response

In [3]:
bcs = [{'variable':0, 'value':0, 'type':'dirichlet',
        'function':lambda X: X[:,1]**3},
        {'variable':0, 'value':0, 'type':'dirichlet',
        'function':lambda X: (tf.constant(1., dtype='float64', shape=(X.shape[0],1))+ X[:,1]**3)*tf.exp(tf.constant(-1., dtype='float64', shape=(X.shape[0],1)))},
        {'variable':1, 'value':0, 'type':'dirichlet',
        'function':lambda X: X[:,0]*tf.exp(-X[:,0])},
        {'variable':1, 'value':1, 'type':'dirichlet',
        'function':lambda X: (X[:,0]+tf.constant(1., dtype='float64', shape=(X.shape[0],1)))*tf.exp(-X[:,0])}]

In [4]:
n_samples = 10
X_train = np.linspace(0, 1, n_samples)
Y_train = np.linspace(0, 1, n_samples)
X_train, Y_train = np.meshgrid(X_train, Y_train)
X_train = X_train.flatten()
Y_train = Y_train.flatten()
samples_train = np.array([X_train, Y_train]).T

In [5]:
n_samples_test = 100
X_test = np.linspace(0, 1, n_samples_test)
Y_test = np.linspace(0, 1, n_samples_test)
X_test, Y_test = np.meshgrid(X_test, Y_test)
X_test = X_test.flatten()
Y_test = Y_test.flatten()
samples_test = np.array([X_test, Y_test]).T

In [6]:
ts = TrialSolution(conditions=bcs, n_i=2, n_h=10, n_o=1, equation_type='PDE', call_method=call)

In [7]:
def diff_loss(network, inputs):
  with tf.GradientTape() as tape2:
    with tf.GradientTape() as tape:
      inputs = tf.convert_to_tensor(inputs)
      tape.watch(inputs)
      tape2.watch(inputs)
      response = network(inputs)  
    grads = tape.gradient(response, inputs)
  laplace = tape2.gradient(grads, inputs)
#   print(grads)
#   print(laplace)
  two = tf.constant(2, dtype='float64')
  pi = tf.constant(np.pi, dtype='float64')
  loss = tf.square(laplace[:,0] + laplace[:,1] + response * grads[:,1]
                   - tf.sin(pi*inputs[:,0])*(two  - pi**2*inputs[:,1]**2 + two * inputs[:,1]**3*tf.sin(pi*inputs[:,0])))
  return loss

### Training

In [8]:
ts.train(X=samples, diff_loss=diff_loss, epochs=10000, message_frequency=1000, optimizer_name='SGD', learning_rate=0.005)

NameError: name 'samples' is not defined

### Plotting the results 

The numerical solution.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
new_shape = int(np.sqrt(samples.shape[0]))
Ze5sol = tf.reshape(ts(samples), shape=(samples.shape[0],)).numpy()
ax.plot_surface(X=samples[:,0].reshape((new_shape, new_shape)), Y=samples[:,1].reshape((new_shape, new_shape)), Z=Ze5sol.reshape((new_shape, new_shape)), label='Numerical - Training')
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
Ze5anal = np.sin(np.pi*samples[:,0])*samples[:,1]**2
Ze5diff = Ze5sol - Ze5anal
ax.plot_surface(X=samples[:,0].reshape((new_shape, new_shape)), Y=samples[:,1].reshape((new_shape, new_shape)), Z=Ze5diff.reshape((new_shape, new_shape)), label='Analytic')
# plt.legend()
plt.show()

In [ ]:
np.abs(Ze5diff**2).mean()

In [ ]:
np.max(np.abs(Ze5diff))